<a href="https://colab.research.google.com/github/elephant-xyz/notebook/blob/feat%2Fcleaning-seeding-step/Mining_Seed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Seed Mining process

In [22]:
# @title Step 1: Upload .env

In [42]:
# @title Step 2: Prepare
# @title  {"vertical-output":true}
import os

parcel_id = "074527L1060260060" # @param {"type":"string"}
address = "114 PAXTON STREET, LEHIGH ACRES, FL 33974" # @param {"type":"string"}
request_method = "GET" # @param {"type":"string"}
url = "https://www.leepa.org/Display/Displayparcel.aspx?folioID=10635895" # @param {"type":"string"}
County = "LEE COUNTY" # @param {"type":"string"}
headers = "" # @param {"type":"string"}

os.environ["parcel_id"] = parcel_id
os.environ["address"] = address
os.environ["request_method"] = request_method
os.environ["url"] = url
os.environ["County"] = County
os.environ["headers"] = headers


import json
import os
import re
import sys
from urllib.parse import urlparse, parse_qs


# Get data from Step 2 environment variables
parcel_id = os.environ.get("parcel_id", "")
address = os.environ.get("address", "")
request_method = os.environ.get("request_method", "")
url = os.environ.get("url", "")
county = os.environ.get("County", "")
headers = os.environ.get("headers", "")


def is_empty_value(value):
    """Check if value is empty or None"""
    if value is None:
        return True
    if isinstance(value, str) and value.strip() == "":
        return True
    return False

def ensure_directory(file_path):
    """Ensure the directory for the file exists"""
    directory = os.path.dirname(file_path)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)

def extract_query_params_and_base_url(url):
    """Extract query parameters and base URL separately"""
    if is_empty_value(url):
        return None, None

    try:
        parsed_url = urlparse(url)

        # Base URL without query parameters
        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"

        # Query parameters as multiValueQueryString format (object of arrays)
        query_params = parse_qs(parsed_url.query)
        multi_value_query = dict(query_params) if query_params else None

        return base_url, multi_value_query
    except Exception as e:
        print(f"Warning: Could not parse URL: {e}")
        return url, None

def create_parcel_folder(parcel_id, address, method, url, county, headers):
    # Create folder name based on parcel_id
    clean_parcel_id = re.sub(r"[^\w\-_]", "_", str(parcel_id))
    folder_name = f"output/{clean_parcel_id}"
    ensure_directory(folder_name + "/")

    # Extract base URL and query parameters separately
    base_url, multi_value_query = extract_query_params_and_base_url(url)

    # Create unnormalized_address.json
    unnormalized_address_data = {
        "full_address": address if not is_empty_value(address) else None,
        "source_http_request": {
            "method": method if not is_empty_value(method) else None,
            "url": base_url if not is_empty_value(base_url) else None,
            "multiValueQueryString": multi_value_query
        },
        "county_jurisdiction": county if not is_empty_value(county) else None,
        "request_identifier": parcel_id if not is_empty_value(parcel_id) else None,
    }
    if headers and not is_empty_value(headers):
        unnormalized_address_data["source_http_request"]["headers"] = headers

    # Create property_seed.json
    property_seed_data = {
        "parcel_id": parcel_id if not is_empty_value(parcel_id) else None,
        "source_http_request": {
            "method": method if not is_empty_value(method) else None,
            "url": base_url if not is_empty_value(base_url) else None,
            "multiValueQueryString": multi_value_query
        },
        "request_identifier": parcel_id if not is_empty_value(parcel_id) else None,
    }
    if headers and not is_empty_value(headers):
        property_seed_data["source_http_request"]["headers"] = headers

    # Create relationship_property_to_address.json
    relationship_data = {
        "from": {"/": "./property_seed.json"},
        "to": {"/": "./unnormalized_address.json"}
    }

    # Create root schema
    root_schema = {
        "label": "Seed",
        "relationships": {"property_seed": {"/": "./relationship_property_to_address.json"}},
    }

    # Write all JSON files
    files_to_create = [
        (f"{folder_name}/unnormalized_address.json", unnormalized_address_data),
        (f"{folder_name}/property_seed.json", property_seed_data),
        (f"{folder_name}/relationship_property_to_address.json", relationship_data),
        (f"{folder_name}/bafkreif7ywbjxu3s6jfi6ginvmsufeux3cd5eujuivg2y7tmqt2qk4rsoe.json", root_schema),
    ]

    for filename, data_obj in files_to_create:
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data_obj, f, indent=2, ensure_ascii=False)

    return folder_name, unnormalized_address_data, property_seed_data

def process_input_data():
    try:
        # Validate required data
        if is_empty_value(parcel_id):
            print("❌ Error: parcel_id is required but not provided")
            return

        # Show extracted URL components
        base_url, multi_value_query = extract_query_params_and_base_url(url)

        # Create parcel folder and files
        folder_name, address_data, property_data = create_parcel_folder(
            parcel_id, address, request_method, url, county, headers
        )

        print(f"\n✅ Prepare done for parcel ID {parcel_id}.")

    except Exception as e:
        print(f"❌ Error processing input data: {e}")
        import traceback
        traceback.print_exc()

# Process the input data
process_input_data()



✅ Prepare done for parcel ID 074527L1060260060.


In [45]:
# @title Step 2: Transform
! pip3 install python-dotenv -q

from dotenv import load_dotenv
load_dotenv()

import subprocess
import sys
import csv


def get_seed_cid_and_html_link(path="upload-results.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        first_row = next(reader, None)
        if first_row is None:
            raise ValueError("CSV file is empty")
        return first_row["dataGroupCid"], first_row["htmlLink"]


def has_submit_errors(path="submit_errors.csv"):
    """
    Повертає True, якщо у файлі submit_errors.csv є хоча б один рядок (після заголовку).
    """
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return next(reader, None) is not None


def run_validate_and_upload():
    try:
        subprocess.run(
            ["npx", "-y", "@elephant-xyz/cli", "validate-and-upload", "output"],
            stdout=subprocess.DEVNULL,    # ховаємо stdout
            stderr=subprocess.PIPE,       # ловимо stderr у буфер
            check=True,
            text=True                     # stderr як рядок
        )
        # Якщо є записані помилки — завершуємо роботу
        if has_submit_errors():
            print("❌ Transform failed, please check submit_errors.csv for details", file=sys.stderr)
            return

        # Інакше — читаємо результати
        seed_group_cid, html_link = get_seed_cid_and_html_link()

        print("✅ Transform done\n")
        print(f"Seed group CID: {seed_group_cid}\n")
        print(f"HTML link: {html_link}")

    except subprocess.CalledProcessError as e:
        # обробка помилок виконання команди
        print(f"Command failed (exit code {e.returncode}):", file=sys.stderr)
        print(e.stderr.strip(), file=sys.stderr)
        sys.exit(e.returncode)


if __name__ == "__main__":
    run_validate_and_upload()


✅ Transform done

Seed group CID: bafkreif7ywbjxu3s6jfi6ginvmsufeux3cd5eujuivg2y7tmqt2qk4rsoe

HTML link: http://dweb.link/ipfs/bafybeiehobmm4zwk2duktd3fpdzwckmtay2s5pzne4sli4jh3upp3lhnzy


In [46]:
# @title Step 3: Validate
! pip3 install python-dotenv -q

from dotenv import load_dotenv
load_dotenv()

import subprocess
import sys
import csv


def get_seed_cid_and_html_link(path="upload-results.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        first_row = next(reader, None)
        if first_row is None:
            raise ValueError("CSV file is empty")
        return first_row["dataGroupCid"], first_row["htmlLink"]


def has_submit_errors(path="submit_errors.csv"):
    """
    Повертає True, якщо у файлі submit_errors.csv є хоча б один рядок (після заголовку).
    """
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return next(reader, None) is not None


def run_validate_and_upload():
    try:
        subprocess.run(
            ["npx", "-y", "@elephant-xyz/cli", "validate-and-upload", "output"],
            stdout=subprocess.DEVNULL,    # ховаємо stdout
            stderr=subprocess.PIPE,       # ловимо stderr у буфер
            check=True,
            text=True                     # stderr як рядок
        )
        # Якщо є записані помилки — завершуємо роботу
        if has_submit_errors():
            print("❌ Validate failed, please check submit_errors.csv for details", file=sys.stderr)
            return

        # Інакше — читаємо результати
        seed_group_cid, html_link = get_seed_cid_and_html_link()
        print("✅ Validate done\n")
        print(f"Seed group CID: {seed_group_cid}\n")
        print(f"HTML link: {html_link}")

    except subprocess.CalledProcessError as e:
        # обробка помилок виконання команди
        print(f"Command failed (exit code {e.returncode}):", file=sys.stderr)
        print(e.stderr.strip(), file=sys.stderr)
        sys.exit(e.returncode)


if __name__ == "__main__":
    run_validate_and_upload()


✅ Validate done

Seed group CID: bafkreif7ywbjxu3s6jfi6ginvmsufeux3cd5eujuivg2y7tmqt2qk4rsoe

HTML link: http://dweb.link/ipfs/bafybeifa4m3lmeyu6adg43iy2d2ywnpnmqgj3mxqnnrbr3bil6dxsulf5u


In [47]:
# @title Step 4: Upload
! pip3 install python-dotenv requests -q

from dotenv import load_dotenv
load_dotenv()

import subprocess
import sys
import csv

import requests


def get_seed_info(path="upload-results.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        first_row = next(reader, None)
        if first_row is None:
            raise ValueError("CSV file is empty")
        return first_row


def has_submit_errors(path="submit_errors.csv"):
    """
    Повертає True, якщо у файлі submit_errors.csv є хоча б один рядок (після заголовку).
    """
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return next(reader, None) is not None


def count_upload_records(path="upload-results.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return sum(1 for _ in reader)


def collect_data_ipfs_links(data_cid):
  seed_data_link = f"https://ipfs.io/ipfs/{data_cid}"

  seed_data = requests.get(seed_data_link).json()

  property_seed_cid = seed_data["relationships"]["property_seed"]["/"]
  property_seed_link = f"https://ipfs.io/ipfs/{property_seed_cid}"

  property_seed_data = requests.get(property_seed_link).json()

  property_cid, address_cid = property_seed_data["from"]["/"], property_seed_data["to"]["/"]

  property_link = f"https://ipfs.io/ipfs/{property_cid}"
  address_link = f"https://ipfs.io/ipfs/{address_cid}"

  return seed_data_link, property_seed_link, property_link, address_link


def run_validate_and_upload():
    try:
        subprocess.run(
            ["npx", "-y", "@elephant-xyz/cli", "validate-and-upload", "output"],
            stdout=subprocess.DEVNULL,    # ховаємо stdout
            stderr=subprocess.PIPE,       # ловимо stderr у буфер
            check=True,
            text=True,
        )

        if has_submit_errors():
            print("❌ Validate failed, please check submit_errors.csv for details", file=sys.stderr)
            return

        seed_info = get_seed_info()
        seed_group_cid, data_cid, html_link = seed_info["dataGroupCid"], seed_info["dataCid"], seed_info["htmlLink"]

        files_uploaded = count_upload_records()

        data_ipfs_links = collect_data_ipfs_links(data_cid)
        seed_group_link, property_seed_link, property_link, address_link = data_ipfs_links

        print("✅ Upload done\n")
        print(f"{files_uploaded} files uploaded\n")

        print(f"Seed group CID: {seed_group_cid}\n")
        print(f"HTML link: {html_link}\n")

        print(f"Seed group IPFS link: {seed_group_link}")
        print(f"relationship IPFS link: {property_seed_link}")
        print(f"property_seed IPFS link: {property_link}")
        print(f"unnormalized_address IPFS link: {address_link}")

    except subprocess.CalledProcessError as e:
        print(f"Command failed (exit code {e.returncode}):", file=sys.stderr)
        print(e.stderr.strip(), file=sys.stderr)
        sys.exit(e.returncode)


if __name__ == "__main__":
    run_validate_and_upload()


✅ Upload done

1 files uploaded

Seed group CID: bafkreif7ywbjxu3s6jfi6ginvmsufeux3cd5eujuivg2y7tmqt2qk4rsoe

HTML link: http://dweb.link/ipfs/bafybeigyo4sndzkpvsgop5l7h6gr3lx7bqrks2ih7tbgfpl5vnhwfr7asu

Seed group IPFS link: https://ipfs.io/ipfs/bafkreigmkzo3bz5rrekjbxglfpgqktgmmgz2awby4o7wmtn3hnnp2xuahu
relationship IPFS link: https://ipfs.io/ipfs/bafkreiewdavv3mmkqbe32t4z5ghjjtzdumdyz3ds6ep7kxacr3rrpoefmq
property_seed IPFS link: https://ipfs.io/ipfs/bafkreib4d6jskeeptsuzerctprbvaqwjnzhqsgulqtwcnb7sley5zgrswa
unnormalized_address IPFS link: https://ipfs.io/ipfs/bafkreihmqhnfv5izziur2ekklnwacg375awzu6ytnco3fkchzzqofu5g7e


In [55]:
# @title Step 5: Submit

! pip3 install python-dotenv -q

from dotenv import load_dotenv
load_dotenv()

import subprocess
import sys
import csv


def get_transaction_hash(path="transaction-status.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        first_row = next(reader, None)
        if first_row is None:
            raise ValueError("CSV file is empty")
        return first_row["transactionHash"]


def has_submit_errors(path="submit_errors.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return next(reader, None) is not None


def run_submit_to_contract():
    try:
        subprocess.run(
            [
                "npx", "-y", "@elephant-xyz/cli", "submit-to-contract", "upload-results.csv",
                "--from-address", "0xefAd08946612A15d5De8D4Db7fc03556b6424075",
                "--api-key", "f7e18cf6-5d07-4e4a-ae23-f27b812614e6",
                "--domain", "oracles-69c46050.staircaseapi.com",
                "--oracle-key-id", "7ad26e0b-67c9-4c2f-95a2-2792c7db5ac7",
            ],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE,
            check=True,
            text=True,
        )
        if has_submit_errors():
            print("❌ Submit failed, please check submit_errors.csv for details", file=sys.stderr)
            return

        transaction_hash = get_transaction_hash()
        transaction_link = f"https://polygonscan.com/tx/{transaction_hash}"

        print("✅ Submit done\n")
        print(f"Transaction link: {transaction_link}")

    except subprocess.CalledProcessError as e:
        print(f"Command failed (exit code {e.returncode}):", file=sys.stderr)
        print(e.stderr.strip(), file=sys.stderr)
        sys.exit(e.returncode)


if __name__ == "__main__":
    run_submit_to_contract()


✅ Submit done

Transaction link: https://polygonscan.com/tx/0x25d6e386ea5a231a9c5a9d30e2193164fe196bdd3c00f35ec8c0d19ac3a9386e


In [56]:
# @title Step 6: Download upload-results.csv